In [9]:
import pandas as pd
from datetime import timedelta


In [10]:
train_df = pd.read_csv('../data/raw/train.csv', parse_dates=['date'])
holiday_df = pd.read_csv('../data/raw/holidays_events.csv', parse_dates=['date'])
oil_df = pd.read_csv('../data/raw/oil.csv', parse_dates=['date'])
stores_df = pd.read_csv('../data/raw/stores.csv')
transactions_df = pd.read_csv('../data/raw/transactions.csv', parse_dates=['date'])
test_df = pd.read_csv('../data/raw/test.csv', parse_dates=['date'])
train_df.shape

(3000888, 6)

In [11]:
# print null rows of any dataframe
def print_null_rows(df, name):
    null_rows = df[df.isnull().any(axis=1)]
    if not null_rows.empty:
        print(f"shape:{df.shape}, Null rows in {name}:\n{null_rows}\n")
    else:
        print(f"shape:{df.shape}, No null rows in {name}.\n")
    
print_null_rows(train_df, 'train_df')
print_null_rows(holiday_df, 'holiday_df')
print_null_rows(oil_df, 'oil_df')
print_null_rows(stores_df, 'stores_df')
print_null_rows(transactions_df, 'transactions_df')
print_null_rows(test_df, 'test_df')

shape:(3000888, 6), No null rows in train_df.

shape:(350, 6), No null rows in holiday_df.

shape:(1218, 2), Null rows in oil_df:
           date  dcoilwtico
0    2013-01-01         NaN
14   2013-01-21         NaN
34   2013-02-18         NaN
63   2013-03-29         NaN
104  2013-05-27         NaN
132  2013-07-04         NaN
174  2013-09-02         NaN
237  2013-11-28         NaN
256  2013-12-25         NaN
261  2014-01-01         NaN
274  2014-01-20         NaN
294  2014-02-17         NaN
338  2014-04-18         NaN
364  2014-05-26         NaN
393  2014-07-04         NaN
434  2014-09-01         NaN
497  2014-11-27         NaN
517  2014-12-25         NaN
522  2015-01-01         NaN
534  2015-01-19         NaN
554  2015-02-16         NaN
588  2015-04-03         NaN
624  2015-05-25         NaN
653  2015-07-03         NaN
699  2015-09-07         NaN
757  2015-11-26         NaN
778  2015-12-25         NaN
783  2016-01-01         NaN
794  2016-01-18         NaN
814  2016-02-15         NaN
84

## 1. Traditional Final Train

In [12]:
# interpolate missing values in oil prices
all_dates = pd.date_range(start=oil_df['date'].min(), end=oil_df['date'].max())
oil_df = oil_df.set_index('date').reindex(all_dates).rename_axis('date').reset_index()
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].interpolate(method='polynomial', order=2)
# fill backward and forward fill for oil prices
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].bfill()

In [13]:
print_null_rows(train_df, 'train_df')
print_null_rows(holiday_df, 'holiday_df')
print_null_rows(oil_df, 'oil_df')
print_null_rows(stores_df, 'stores_df')
print_null_rows(transactions_df, 'transactions_df')
print_null_rows(test_df, 'test_df')

shape:(3000888, 6), No null rows in train_df.

shape:(350, 6), No null rows in holiday_df.

shape:(1704, 2), No null rows in oil_df.

shape:(54, 5), No null rows in stores_df.

shape:(83488, 3), No null rows in transactions_df.

shape:(28512, 5), No null rows in test_df.



In [14]:
# create faetures for train and test sets
def create_features(df):
    def days_since_payday(date):
        day = date.day
        if day <= 15:
            # Days since last month's end
            last_month_end = date.replace(day=1) - timedelta(days=1)
            return (date - last_month_end).days
        else:
            # Days since 15th of current month
            current_month_15th = date.replace(day=15)
            return (date - current_month_15th).days
        
    def days_until_payday(date):
        day = date.day
        if day < 15:
            # Days until 15th
            return 15 - day
        else:
            # Days until month end
            next_month = date.replace(day=28) + timedelta(days=4)
            month_end = next_month - timedelta(days=next_month.day)
            return (month_end - date).days
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['weekofyear'] = df['date'].dt.isocalendar().week
    df['day_of_year'] = df['date'].dt.dayofyear
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    df['is_quarter_start'] = df['date'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['date'].dt.is_quarter_end.astype(int)
    df['is_payday'] = ((df['day'] == 15) | df['date'].dt.is_month_end).astype(int)
    df['days_since_payday'] = df['date'].apply(days_since_payday)
    df['days_until_payday'] = df['date'].apply(days_until_payday)
    return df

def create_lag_features(df, target_col='sales', lags=[1, 7, 14, 30]):
    """Create lag features for time series"""
    df_sorted = df.sort_values(['store_nbr', 'family', 'date'])
    
    for lag in lags:
        df_sorted[f'{target_col}_lag_{lag}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].shift(lag)
    
    return df_sorted

def create_rolling_features(df, target_col='sales', windows=[7, 14, 30]):
    """Create rolling window statistics"""
    df_sorted = df.sort_values(['store_nbr', 'family', 'date'])
    
    for window in windows:
        # Rolling mean
        df_sorted[f'{target_col}_rolling_mean_{window}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        
        # Rolling std
        df_sorted[f'{target_col}_rolling_std_{window}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
        
        # Rolling max
        df_sorted[f'{target_col}_rolling_max_{window}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).max()
        )
        
        # Rolling min
        df_sorted[f'{target_col}_rolling_min_{window}'] = df_sorted.groupby(['store_nbr', 'family'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).min()
        )
    
    return df_sorted


In [15]:
train_df = create_features(train_df)
print_null_rows(train_df, 'train_df_temporal')

shape:(3000888, 20), No null rows in train_df_temporal.



In [16]:
train_df = create_rolling_features(train_df)
# print number of nulls in train_df_rolling
train_df.isnull().sum()

id                          0
date                        0
store_nbr                   0
family                      0
sales                       0
onpromotion                 0
year                        0
month                       0
day                         0
dayofweek                   0
weekofyear                  0
day_of_year                 0
is_weekend                  0
is_month_start              0
is_month_end                0
is_quarter_start            0
is_quarter_end              0
is_payday                   0
days_since_payday           0
days_until_payday           0
sales_rolling_mean_7        0
sales_rolling_std_7      1782
sales_rolling_max_7         0
sales_rolling_min_7         0
sales_rolling_mean_14       0
sales_rolling_std_14     1782
sales_rolling_max_14        0
sales_rolling_min_14        0
sales_rolling_mean_30       0
sales_rolling_std_30     1782
sales_rolling_max_30        0
sales_rolling_min_30        0
dtype: int64

In [17]:
train_df = create_lag_features(train_df)
train_df.isnull().sum()

id                           0
date                         0
store_nbr                    0
family                       0
sales                        0
onpromotion                  0
year                         0
month                        0
day                          0
dayofweek                    0
weekofyear                   0
day_of_year                  0
is_weekend                   0
is_month_start               0
is_month_end                 0
is_quarter_start             0
is_quarter_end               0
is_payday                    0
days_since_payday            0
days_until_payday            0
sales_rolling_mean_7         0
sales_rolling_std_7       1782
sales_rolling_max_7          0
sales_rolling_min_7          0
sales_rolling_mean_14        0
sales_rolling_std_14      1782
sales_rolling_max_14         0
sales_rolling_min_14         0
sales_rolling_mean_30        0
sales_rolling_std_30      1782
sales_rolling_max_30         0
sales_rolling_min_30         0
sales_la

In [19]:
train_df.dropna(inplace=True)
print_null_rows(train_df, 'train_df_final')

shape:(2947428, 36), No null rows in train_df_final.



In [20]:
national_holidays = holiday_df[holiday_df['locale'] == 'National']['date'].unique()
regional_holidays = holiday_df[holiday_df['locale'] == 'Regional']['date'].unique()
local_holidays = holiday_df[holiday_df['locale'] == 'Local']['date'].unique()
additional_holidays = holiday_df[holiday_df['type'] == 'Additional']['date'].unique()
working_days = holiday_df[holiday_df['type'] == 'Work Day']['date'].unique()
events = holiday_df[holiday_df['type'] == 'Event']['date'].unique()
bridge_days = holiday_df[holiday_df['type'] == 'Bridge']['date'].unique()
transsferred_days = holiday_df[holiday_df['transferred'] == True]['date'].unique()

In [21]:
# add holiday features to train
def add_holiday_features(df):
    df['is_national_holiday'] = df['date'].isin(national_holidays).astype(int)
    df['is_regional_holiday'] = df['date'].isin(regional_holidays).astype(int)
    df['is_local_holiday'] = df['date'].isin(local_holidays).astype(int)
    df['is_additional_holiday'] = df['date'].isin(additional_holidays).astype(int)
    df['is_working_day'] = df['date'].isin(working_days).astype(int)
    df['is_event'] = df['date'].isin(events).astype(int)
    df['is_bridge_day'] = df['date'].isin(bridge_days).astype(int)
    df['is_transferred_day'] = df['date'].isin(transsferred_days).astype(int)
    return df

# create features for train and test sets
train_df = add_holiday_features(train_df)
print_null_rows(train_df, 'train_df')

shape:(2947428, 44), No null rows in train_df.



In [ ]:
train_df = train_df.merge(oil_df, on='date', how='left')
print_null_rows(train_df, 'train_df_final')

In [ ]:
train_df = train_df.merge(stores_df, on='store_nbr', how='left')
print_null_rows(train_df, 'train_df_final')

In [ ]:
train_df = train_df.merge(transactions_df, on=['date', 'store_nbr'], how='left')
print_null_rows(train_df, 'train_df_final')

shape:(2947428, 50), Null rows in train_df_final:
              id       date  store_nbr      family  sales  onpromotion  year  \
334       648648 2014-01-01          1  AUTOMOTIVE    0.0            0  2014   
698      1297296 2015-01-01          1  AUTOMOTIVE    0.0            0  2015   
885      1630530 2015-07-07          1  AUTOMOTIVE    0.0            0  2015   
1062     1945944 2016-01-01          1  AUTOMOTIVE    0.0            0  2016   
1063     1947726 2016-01-02          1  AUTOMOTIVE    7.0            0  2016   
...          ...        ...        ...         ...    ...          ...   ...   
2946472  1298945 2015-01-01         54     SEAFOOD    0.0            0  2015   
2946836  1947593 2016-01-01         54     SEAFOOD    0.0            0  2016   
2946838  1951157 2016-01-03         54     SEAFOOD    2.0            0  2016   
2946839  1952939 2016-01-04         54     SEAFOOD    3.0            0  2016   
2947201  2598023 2017-01-01         54     SEAFOOD    0.0            0

In [27]:
# fill missing values in transactions with 0
train_df['transactions'] = train_df['transactions'].fillna(0)
print_null_rows(train_df, 'train_df_final')

shape:(2947428, 50), No null rows in train_df_final.



In [ ]:
train_df.to_csv('../data/interim/traditional_final_train.csv', index=False)